## Process

Whenever we use a computer, we are always interacting with programs like opening a browser, wrting on a word processor, or using command line interfaces. The operating system's abstraction for running a program is a process. These processes are both parent and child processes.

The Python subprocess module is used to launch child processes. Here, Python is the parent process and child could be anything from shell to the GUI applications.

Let's create an example. In this example we will create a simple Python program and we will use subprocess to call it. Although we don't need to call a Python program as a separate sub-process, it can as well be imported as a module but the main idea is to show how subprocess module be cross-platform in nature. 

The Python program will add the numbers with sleep time of 1 sec and print them. (A simple and a cool program)

In [3]:
import subprocess

In [14]:
subprocess.run(["python3", "adder_timer.py", "3"], check = True, timeout = 10)

CompletedProcess(args=['python3', 'adder_timer.py', '3'], returncode=0)

On executing run(), the program runs in the terminal (from where you have launched in the sub-process) and once it is done it returns an instance of CompletedProcess class with a returncode = 0. If there is some errors the resturncode is not equal to 0. Under the hood, run() makes a system call and it does not need a shell to do it.

With subprocess we can run any App like say, notepad. The following command opens up a notepad and when you close it, it again returns an instance of CompletedProcess class with a returncode = 0.

So, by default many modules (higher level of abstarction) uses subprocess() to do low-level operations.

* Exceptions Handling-
1. check = True raises the exception when returncode is not equals to 0. (CalledProcessError)
2. Some processes might take too long to run or may have hang indefinitely (TimeoutExpired). Giving a timeout parameter will end the sub-process. It is in seconds
3. FileNotFoundError if there is no file found, in our case say the adder_timer.py does not exist.

In [9]:
subprocess.run(['notepad'])

CompletedProcess(args=['notepad'], returncode=0)

#### Exception Handling

In [17]:
try:
    subprocess.run(
        ["python3", "adder_timer.py", "5"], timeout = 10
    )
except FileNotFoundError as exc:
    print(f"Process failed to locate the executable file.\n{exc}")
except subprocess.CalledProcessError as exc:
    print(f"Process failed to return a success returncode"
         f"The process returned {exc.returncode}\n{exc}"
         )
except subprocess.TimeoutExpired as exc:
    print(f"Process timed out.\n{exc}")